In [61]:
import pandas as pd

In [ ]:
"""from sklearn.model_selection import train_test_split
Y=data['Hallucination']
X=data.drop(['Hallucination','Prediction'],axis=1)
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.33,random_state=42)
Y_train"""

In [ ]:
"""from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# Load the data
data = pd.read_csv('/content/Hallucination-Dataset-400-Samples.csv')
data['Question'].fillna('', inplace=True)
# Create input features and target variable
X = data[['Context', 'Question', 'Answer']]
y = data['Hallucination']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions and fill the prediction column
predictions = clf.predict(X_test)
data['prediction'] = predictions

# Evaluate the model
print(classification_report(y_test, predictions))"""

In [ ]:
"""from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
import math
num_epochs = 5
model_save_path = "./model_dump"
model_name = 'cross-encoder/nli-distilroberta-base'
model = CrossEncoder(model_name, num_labels=1, automodel_args={'ignore_mismatched_sizes':True})


train_examples, test_examples = X_train.values.tolist(), Y_train.values.tolist()

train_examples = []
for i, row in X_train.iterrows():
    train_examples.append(InputExample(texts=[row['Context'], row['Question'],row['Answer']], label=Y_train[i]))
#for i, row in X_train.iterrows():
    #train_examples.append(InputExample(texts=[row['Context'], row['Question'],row['Answer']], label=Y_train))
test_examples = []
for i, row in X_test.iterrows():
    test_examples.append(InputExample(texts=[row['Context'], row['Question'], row['Answer']], label=Y_test[i]))
#for i, row in X_test.iterrows():
    #test_examples.append(InputExample(texts=[row['Context'], row['Question'],row['Answer']], label=Y_train))
test_evaluator = CEBinaryClassificationEvaluator.from_input_examples(test_examples, name='test_eval')

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_dataloader=train_dataloader,
          evaluator=test_evaluator,
          epochs=num_epochs,
          evaluation_steps=10_000,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          show_progress_bar=True)
"""

In [62]:
f=input("Enter a file path: ")

Enter a file path: /content/Hallucination-Dataset-400-Samples.csv


In [63]:
data=pd.read_csv(f)

In [64]:
!pip install sentence_transformers

In [65]:
from sentence_transformers import CrossEncoder
inp = data.apply(lambda row: [list(row[['Context', 'Question', 'Answer']])], axis=1)
if inp.dtypes != 'str':
    inp = inp.apply(lambda x: [str(item) for item in x])
model = CrossEncoder('vectara/hallucination_evaluation_model')
scores = model.predict(inp)

In [66]:
print(len(scores))

402


In [67]:
Prediction=pd.DataFrame(scores)

In [68]:
Prediction

,0
0,0.225210
1,0.297898
2,0.468616
3,0.743687
4,0.446383
...,...
397,0.998542
398,0.966729
399,0.998080
400,0.387133


In [69]:
Prediction=Prediction[0].map(lambda x:1 if x<0.5 else 0)

In [70]:
Prediction

0      1
1      1
2      1
3      0
4      1
      ..
397    0
398    0
399    0
400    1
401    1
Name: 0, Length: 402, dtype: int64

In [71]:
data

,Context,Question,Answer,Hallucination,Prediction
0,The revolutionary CoolMOS™ power MOSFET family...,What is the purpose of Infineon’s Engine Manag...,Minimizing layout resistance and maximizing co...,1,NaN
1,Magnetic switches and magnetic latches operate...,What scenario did the algorithms generate?,There are no different types of magnetic senso...,1,NaN
2,Serial Flash Discoverable Parameters (SFDP) is...,What is the standard number for Serial Flash D...,The standard number for SFDP is JESD2163,0,NaN
3,"EiceDRIVERTM 2EDN family is known for fast, pr...",What makes the 2EDN family a new reference in ...,The 2EDN family features 4 V and 8 V UVLO for ...,0,NaN
4,The ICL88xx family of single-stage flyback con...,How many I/O lanes does Dual Quad I/O SPI supp...,Energy harvesting,1,NaN
...,...,...,...,...,...
397,While exploring AI-driven applications in semi...,What are the benefits of a consistent pinout s...,The AI system projected a scenario where semic...,1,NaN
398,The 2ED132x family consists of four 1200-V SOI...,"What does the 2ED132x family offer, and what a...",Four 1200-V SOI gate drivers with 2.3 A output...,0,NaN
399,our complete solutions include all the buildin...,How does your solution contribute to faster ma...,Our solution accelerates market entry by provi...,0,NaN
400,6EDL7141 is Infineon’s latest 3-phase motor c...,What are the ideal applications for the 6EDL71...,Suitable for underwater applications.,1,NaN


In [72]:
data.drop("Prediction", axis=1)

,Context,Question,Answer,Hallucination
0,The revolutionary CoolMOS™ power MOSFET family...,What is the purpose of Infineon’s Engine Manag...,Minimizing layout resistance and maximizing co...,1
1,Magnetic switches and magnetic latches operate...,What scenario did the algorithms generate?,There are no different types of magnetic senso...,1
2,Serial Flash Discoverable Parameters (SFDP) is...,What is the standard number for Serial Flash D...,The standard number for SFDP is JESD2163,0
3,"EiceDRIVERTM 2EDN family is known for fast, pr...",What makes the 2EDN family a new reference in ...,The 2EDN family features 4 V and 8 V UVLO for ...,0
4,The ICL88xx family of single-stage flyback con...,How many I/O lanes does Dual Quad I/O SPI supp...,Energy harvesting,1
...,...,...,...,...
397,While exploring AI-driven applications in semi...,What are the benefits of a consistent pinout s...,The AI system projected a scenario where semic...,1
398,The 2ED132x family consists of four 1200-V SOI...,"What does the 2ED132x family offer, and what a...",Four 1200-V SOI gate drivers with 2.3 A output...,0
399,our complete solutions include all the buildin...,How does your solution contribute to faster ma...,Our solution accelerates market entry by provi...,0
400,6EDL7141 is Infineon’s latest 3-phase motor c...,What are the ideal applications for the 6EDL71...,Suitable for underwater applications.,1


In [73]:
data['Prediction']=Prediction

In [74]:
data

,Context,Question,Answer,Hallucination,Prediction
0,The revolutionary CoolMOS™ power MOSFET family...,What is the purpose of Infineon’s Engine Manag...,Minimizing layout resistance and maximizing co...,1,1
1,Magnetic switches and magnetic latches operate...,What scenario did the algorithms generate?,There are no different types of magnetic senso...,1,1
2,Serial Flash Discoverable Parameters (SFDP) is...,What is the standard number for Serial Flash D...,The standard number for SFDP is JESD2163,0,1
3,"EiceDRIVERTM 2EDN family is known for fast, pr...",What makes the 2EDN family a new reference in ...,The 2EDN family features 4 V and 8 V UVLO for ...,0,0
4,The ICL88xx family of single-stage flyback con...,How many I/O lanes does Dual Quad I/O SPI supp...,Energy harvesting,1,1
...,...,...,...,...,...
397,While exploring AI-driven applications in semi...,What are the benefits of a consistent pinout s...,The AI system projected a scenario where semic...,1,0
398,The 2ED132x family consists of four 1200-V SOI...,"What does the 2ED132x family offer, and what a...",Four 1200-V SOI gate drivers with 2.3 A output...,0,0
399,our complete solutions include all the buildin...,How does your solution contribute to faster ma...,Our solution accelerates market entry by provi...,0,0
400,6EDL7141 is Infineon’s latest 3-phase motor c...,What are the ideal applications for the 6EDL71...,Suitable for underwater applications.,1,1


In [75]:
data.to_csv("/content/predicted.csv")